# ReMePARK — Análisis Longitudinal de MDS-UPDRS

Laboratorio Clínico de Enfermedades Neurodegenerativas (LCEN)  
Instituto Nacional de Neurología y Neurocirugía “Manuel Velasco Suárez” (INNN)  
Ciudad de México, México


## Descripción General

Este cuaderno analiza datos longitudinales de la escala **MDS-UPDRS** (Movement Disorder Society — Unified Parkinson’s Disease Rating Scale) de la cohorte **ReMePARK**.  
El objetivo es visualizar trayectorias individuales, estimar cambios a lo largo del tiempo y explorar asociaciones con variables clínicas y terapéuticas.


## Objetivos del Análisis

1. Calcular el puntaje total de la MDS-UPDRS a partir de las subescalas I–IV.  
2. Describir la evolución temporal del puntaje total y de cada subescala.  
3. Comparar trayectorias entre grupos (por ejemplo, estado ON/OFF o cohorte intervención/control).  
4. Modelar el cambio longitudinal mediante regresiones lineales o modelos mixtos simples.  
5. Generar visualizaciones reproducibles para informes y publicaciones.


## Dependencias y entorno

Requiere Python ≥ 3.9 y los siguientes paquetes:

- `pandas`, `numpy`
- `matplotlib`, `seaborn`
- `statsmodels` (para análisis estadístico)

Instalación rápida:
```bash
pip install -r requirements.txt


| Artículo                                                                                                   | Objetivo y muestra                                               | Métodos clave                                                         | Hallazgos principales                                                                                                                                                                            | Relevancia práctica                                                                                                       |
| ---------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------- | --------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------- |
| *Progression of MDS-UPDRS Scores Over Five Years in De Novo Parkinson Disease*                             | 362 pacientes PPMI sin tratamiento al inicio; seguimiento 5 años | Modelos lineales mixtos (OFF)                                         | Progresión media anual: Total + 4.7; Parte I + 0.92; Parte II + 0.99; Parte III + 2.4 puntos                                                                                                     | Sirve de referencia para comparar eficacia de terapias modificadoras y para asesorar a pacientes recién diagnosticados.   |
| *Modeling of PD Progression and Implications for Detection of Disease Modification in Treatment Trials*    | Datos PPMI; validación con placebo de PASADENA                   | Modelización no lineal de efectos mixtos + simulaciones de ensayos    | Parte III progresa ≈3 pt/año, tres veces más rápido que Parte II/I (≈1 pt/año). Un efecto DMT medible en Parte III puede detectarse en ≤2 años; Parte II requiere ≥3–5 años.                     | Ofrece una herramienta para dimensionar ensayos y seleccionar el sub-score más sensible según la duración.                |
| *Estimation of and Clinical Consensus on the Meaningful Motor Progression Threshold on MDS-UPDRS Part III* | Análisis ancla-basado (PASADENA) + panel Delphi (13 expertos)    | Comparación con CGI-I y consenso iterativo                            | Cambio de +5 puntos en Parte III (OFF) marca progresión motora clínicamente significativa.                                                                                                       | Establece un umbral único para eventos de progresión y desenlaces “time-to-event” en ensayos precoces.                    |
| *Handling Missing Values in the MDS-UPDRS*                                                                 | 842 pacientes (validación) + cohorte internacional               | Eliminación secuencial de ítems; prorrateo; concordancia de Lin ≥0.95 | Se permite: Parte I ≤1 ítem, Parte II ≤2 (aleatorios) o 1 (fijos), Parte III ≤7 (aleatorios) o 3 (fijos), Parte IV ningún ítem; por encima, descartar o imputar con métodos más robustos.        | Proporciona reglas operativas para limpiar bases de datos o aplicar telemedicina sin comprometer la validez del puntaje.  |
| *MCID para MDS-UPDRS Partes I y II*                                                                        | 365 pacientes, 985 visitas; PGI-I como ancla                     | Tres técnicas convergentes (regresión ordinal, ROC, medias)           | Límite de importancia mínima clínica (IMC): Parte I ±2.5–2.6 pts (mejoría/empeoramiento); Parte II ±3.0–2.5 pts; Combinado I+II ±5.7–4.7 pts.                                                    | Permite interpretar cambios percibidos por el paciente en actividades de la vida diaria y planificar tamaños muestrales.  |
| *MCID para MDS-UPDRS Parte III*                                                                            | 260 pacientes, 728 visitas; CGI-I como ancla                     | Métodos ancla y distribución                                          | IMC asimétrica: −3.25 pts (mejoría mínima), +4.63 pts (empeoramiento mínimo) en Parte III.                                                                                                       | Útil para decidir significación clínica de cambios motores en práctica y ensayos.                                         |
| *Systematic Review of MCID Thresholds in Movement Disorders*                                               | 32 estudios (11 sobre UPDRS/MDS-UPDRS)                           | Revisión PRISMA + evaluación de sesgo                                 | Confirma rangos: Parte I ±2.5 pts; Parte II ±3 pts; Parte III −3 a −5 pts (mejoría), +4–5 pts (empeoramiento); Parte IV ±0.9 pts. Resalta variabilidad metodológica y necesidad de estandarizar. | Ofrece marco comparativo y resalta la importancia de reportar MCID junto a significación estadística.                     |


| Parte MDS-UPDRS                 | Leve ↔ Moderada | Moderada ↔ Grave |
| ------------------------------- | --------------- | ---------------- |
| **I** (no-motor EDV)            | **10/11**       | **21/22**        |
| **II** (motor EDV)              | **12/13**       | **29/30**        |
| **III** (exploración)           | **32/33**       | **58/59**        |
| **IV** (complicaciones motoras) | **4/5**         | **12/13**        |


## 🔗 Montar Google Drive

Para acceder a los archivos del proyecto almacenados en **Google Drive**, monta tu unidad de forma segura.  
Esto permite leer y guardar datos directamente desde tu repositorio de Drive sin descargarlos manualmente.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## 🧩 Importar todas las librerías

En esta sección se importan todas las librerías necesarias para el procesamiento, análisis y visualización de los datos del proyecto **ReMePARK – MDS-UPDRS*



In [ ]:
# Manejo de datos
import pandas as pd
import numpy as np

# Fechas y tiempo
from datetime import datetime

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Manejo de archivos y rutas
import glob
import os
import re

# Configuración general de las gráficas
sns.set(style='whitegrid', context='notebook', font_scale=1.2)
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12


### 5. **Carga de datos**
```markdown
## Carga y estructura de los datos

El archivo de entrada debe colocarse en la carpeta `data/` (no incluido en el repositorio por motivos de privacidad).

### Formato esperado (`rempark_mdsupdrs_long.csv`)
| Columna | Tipo | Descripción |
|----------|------|-------------|
| `SubjectID` | str/int | Identificador del participante |
| `Visit` | str/int | Número o etiqueta de visita |
| `VisitDate` | date | Fecha de la visita |
| `MDS_UPDRS_I` | float | No-motor experiences of daily living |
| `MDS_UPDRS_II` | float | Motor experiences of daily living |
| `MDS_UPDRS_III` | float | Exploración motora |
| `MDS_UPDRS_IV` | float | Complicaciones motoras |
| `LEDD` | float (opcional) | Dosis equivalente de levodopa |
| `State` | str (opcional) | Estado ON/OFF |
| `Group` | str (opcional) | Cohorte o intervención |


In [ ]:
# Definir ruta
file_path = '/content/drive/MyDrive/LCEN/08_Bases de Datos y Herramientas/08.1_ReMePARK/08.1.6_Unified ReMePARK 2024/Remepark_MDS-UPDRS.xlsx'

# Cargar archivo Excel
df = pd.read_excel(file_path)

# Ver las primeras filas
df.head()

## 💾 Convertir a CSV

Una vez que los datos han sido limpiados o combinados, se pueden exportar a formato **CSV** para conservar una versión procesada o facilitar su análisis posterior.

In [ ]:
# Definir nombre y ruta del archivo de salida
csv_path = '/content/drive/MyDrive/LCEN/08_Bases de Datos y Herramientas/08.1_ReMePARK/08.1.6_Unified ReMePARK 2024/Remepark_mdsupdrs.csv'

# Exportar DataFrame a CSV
df.to_csv(csv_path, index=False)

print(f"Archivo guardado correctamente en: {output_path}")


In [ ]:
df = pd.read_csv(csv_path)


## 🧮 Inicializar el marco de resultados de validación

Antes de comenzar la validación de los cálculos o comparaciones entre columnas, es útil crear un **marco de resultados** (DataFrame) que almacene los indicadores de validación para cada fila o sujeto.  

Esto facilita centralizar los resultados de verificación, métricas o discrepancias detectadas en los cálculos del MDS-UPDRS.

In [ ]:
# Crear marco de resultados vacío
validation_results_updrs = pd.DataFrame()

# Copiar identificadores desde el DataFrame original
validation_results_updrs["fila_id"] = df["fila_id"]


## ✅ Validación `num.consec`

Esta sección verifica la **integridad y consistencia** de la columna `num.consec`, la cual representa un identificador o número consecutivo único para cada registro.  
Las validaciones comprueban tres aspectos fundamentales:

1. **No nulos** → garantiza que todos los registros tengan valor.  
2. **Tipo numérico** → asegura que los valores sean enteros o flotantes.  
3. **Unicidad** → valida que no existan duplicados en esta columna.

In [ ]:
# Validaciones para num.consec
validation_results_updrs["num.consec.notnull"] = df["num.consec"].notnull()
validation_results_updrs["num.consec.int"] = df["num.consec"].apply(lambda x: isinstance(x, (int, float)) and float(x).is_integer())
validation_results_updrs["num.consec.unique"] = ~df.duplicated(subset=["num.consec"], keep=False)


In [ ]:
(validation_results_updrs[["num.consec.notnull", "num.consec.int", "num.consec.unique"]] == False).sum()


In [ ]:
# Mostrar filas que fallan al menos una validación de num.consec
errores_num_consec = ~validation_results_updrs[["num.consec.notnull", "num.consec.int", "num.consec.unique"]].all(axis=1)

# Mostrar datos originales correspondientes
df[errores_num_consec]


## 🧠 Validación de variables clínicas — Subescalas MDS-UPDRS

En esta sección se validan las **subescalas clínicas** de la MDS-UPDRS (Partes I a IV) para garantizar la calidad y consistencia de los datos antes del análisis estadístico.

Las verificaciones contemplan:

1. **Presencia de datos** (`notnull`)  
2. **Tipo numérico válido** (`isinstance`)  
3. **Valores dentro de los rangos esperados** (según el diseño de la escala)

Notas:

Cada parte de la MDS-UPDRS tiene un rango máximo distinto, pero en esta validación inicial se aplica un rango general (0–132) como filtro de integridad.

Si se desea una validación más específica, se pueden ajustar los límites:

Parte I: 0–52

Parte II: 0–52

Parte III: 0–132

Parte IV: 0–24

Los resultados (True/False) se almacenan por fila, permitiendo identificar rápidamente registros problemáticos.

## 🧩 Identificar columnas por parte del MDS-UPDRS

Antes de realizar validaciones o cálculos, es necesario **identificar las columnas** correspondientes a cada una de las partes de la escala **MDS-UPDRS (I, II, III, IV)**.  
Esto facilita aplicar operaciones agrupadas (por ejemplo, sumas o validaciones de rango) sin escribir manualmente cada nombre de ítem.


Notas:

Cada lista contiene las columnas correspondientes a los ítems de cada parte.

El prefijo ("UPDRS1.", "UPDRS2.", etc.) debe coincidir exactamente con los nombres de tus columnas.

Este enfoque automatiza la selección, útil cuando se trabaja con bases de datos extensas o exportadas desde REDCap, Excel o SPSS.

In [ ]:
updrs1_cols = [col for col in df.columns if col.startswith("UPDRS1.")]
updrs2_cols = [col for col in df.columns if col.startswith("UPDRS2.")]
updrs3_cols = [col for col in df.columns if col.startswith("UPDRS3.")]
updrs4_cols = [col for col in df.columns if col.startswith("UPDRS4.")]

## ✅ Validación de columnas con valores numéricos

Una vez identificadas las columnas, se valida que **todas las celdas contengan valores numéricos válidos** en cada parte de la MDS-UPDRS.  
Esto previene errores en los cálculos de sumas o promedios en etapas posteriores.

Notas:

pd.to_numeric(..., errors="coerce") convierte valores no numéricos en NaN, evitando que se rompan las operaciones aritméticas.

Es recomendable verificar después el número de NaN generados para detectar errores de carga o formato.

In [ ]:
df[updrs1_cols + updrs2_cols + updrs3_cols + updrs4_cols] = df[updrs1_cols + updrs2_cols + updrs3_cols + updrs4_cols].apply(pd.to_numeric, errors="coerce")

## 📚 Agrupar y validar MDS-UPDRS por partes (I–IV)

En esta sección se:
1) **Identifican** las columnas de cada parte de la MDS-UPDRS.  
2) **Convierten** a numérico con coerción (valores inválidos → `NaN`).  
3) **Validan rangos por fila** (ítems con valores entre 0 y 4).  
4) **Filtran** filas válidas por parte para evitar sesgos en sumas y promedios.

> **Regla de validación:** cada fila es válida si **todos** los ítems no nulos de esa parte están en \[0, 1, 2, 3, 4\].

Notas
- **Conversión a numérico:** asegura que valores como `"3 "` o `"2.0"` no provoquen errores (se convierten a `float`).  
- **Validación 0–4:** sigue la escala de cada ítem MDS-UPDRS (0 = normal; 4 = severo).  
- **Filtrado incremental:** se filtra por parte de forma secuencial para conservar observaciones con datos válidos en todas las partes consideradas.  
- **Suma por parte:** usa `min_count=1` para evitar que filas con todos `NaN` devuelvan 0 de forma engañosa.  
- **Ajusta las listas de columnas** si tus nombres no coinciden exactamente con los ejemplos (`UPDRS1.x`, `UPDRS2.x`, etc.).  

In [ ]:
# ===============================
# MDS-UPDRS Part 1️⃣
# ===============================
updrs1_cols = [f"UPDRS1.{i}" for i in range(1, 14)]  # UPDRS1.1 to UPDRS1.13

df_updrs1 = df.copy()
df_updrs1[updrs1_cols] = df_updrs1[updrs1_cols].apply(pd.to_numeric, errors="coerce")

df_updrs1_filtered = df_updrs1[df_updrs1[updrs1_cols].apply(
    lambda row: row.dropna().isin([0, 1, 2, 3, 4]).all(), axis=1
)]

# ===============================
# MDS-UPDRS Part 2️⃣
# ===============================
updrs2_cols = [f"UPDRS2.{i}" for i in range(1, 14)]  # UPDRS2.1 to UPDRS2.13

df_updrs2 = df_updrs1_filtered.copy()
df_updrs2[updrs2_cols] = df_updrs2[updrs2_cols].apply(pd.to_numeric, errors="coerce")

df_updrs2_filtered = df_updrs2[df_updrs2[updrs2_cols].apply(
    lambda row: row.dropna().isin([0, 1, 2, 3, 4]).all(), axis=1
)]

# ===============================
# MDS-UPDRS Part 3️⃣
# ===============================
updrs3_cols = [
    "UPDRS3.1", "UPDRS3.2", "UPDRS3.3C", "UPDRS3.3MSD", "UPDRS3.3MSI", "UPDRS3.3MID", "UPDRS3.3MII",
    "UPDRS3.4MD", "UPDRS3.4MI", "UPDRS3.5MD", "UPDRS3.5MI", "UPDRS3.6MD", "UPDRS3.6MI",
    "UPDRS3.7PD", "UPDRS3.7PI", "UPDRS3.8PD", "UPDRS3.8PI", "UPDRS3.9", "UPDRS3.10",
    "UPDRS3.11", "UPDRS3.12", "UPDRS3.13", "UPDRS3.14", "UPDRS3.15MD", "UPDRS3.15MI",
    "UPDRS3.16MD", "UPDRS3.16MI", "UPDRS3.17MSD", "UPDRS3.17MSI", "UPDRS3.17MID",
    "UPDRS3.17MII", "UPDRS3.17LM", "UPDRS3.18"
]

df_updrs3 = df_updrs2_filtered.copy()
df_updrs3[updrs3_cols] = df_updrs3[updrs3_cols].apply(pd.to_numeric, errors="coerce")

df_updrs3_filtered = df_updrs3[df_updrs3[updrs3_cols].apply(
    lambda row: row.dropna().isin([0, 1, 2, 3, 4]).all(), axis=1
)]

# ===============================
# MDS-UPDRS Part 4️⃣
# ===============================
updrs4_cols = [f"UPDRS4.{i}" for i in range(1, 7)]  # UPDRS4.1 to UPDRS4.6

df_updrs4 = df_updrs3_filtered.copy()
df_updrs4[updrs4_cols] = df_updrs4[updrs4_cols].apply(pd.to_numeric, errors="coerce")

df_updrs4_filtered = df_updrs4[df_updrs4[updrs4_cols].apply(
    lambda row: row.dropna().isin([0, 1, 2, 3, 4]).all(), axis=1
)]


## Calcular las puntuaciones totales de cada parte

Se suma por fila cada conjunto de ítems pertenecientes a la parte I–IV.  
`skipna=True` evita que valores faltantes rompan la suma; `min_count=1` impide que una fila con todos `NaN` devuelva 0.

Asegúrate de que updrs1_cols, updrs2_cols, updrs3_cols y updrs4_cols contengan únicamente los ítems válidos de cada parte.

Si decides imputar valores faltantes antes de sumar, hazlo explícito (por ejemplo, fillna(0)), y documenta la decisión.

In [ ]:
df["UPDRS1.TOTAL"] = df[updrs1_cols].sum(axis=1, skipna=True)
df["UPDRS2.TOTAL"] = df[updrs2_cols].sum(axis=1, skipna=True)
df["UPDRS3.TOTAL"] = df[updrs3_cols].sum(axis=1, skipna=True)
df["UPDRS4.TOTAL"] = df[updrs4_cols].sum(axis=1, skipna=True)


## Calcular la puntuación total MDS-UPDRS

La puntuación total se obtiene como la suma de las partes I–IV.  
Se recomienda usar `min_count=1` en cada parcial (como arriba) para que la suma total refleje correctamente filas con datos faltantes.


Si tu base no incluye la parte IV, ajusta la suma en consecuencia.

Verifica rangos esperados post-suma para detectar valores atípicos por carga/transformación errónea.

In [ ]:
df["UPDRS.TOTAL"] = (
    df["UPDRS1.TOTAL"] +
    df["UPDRS2.TOTAL"] +
    df["UPDRS3.TOTAL"] +
    df["UPDRS4.TOTAL"]
)

In [ ]:
print(df[["UPDRS1.TOTAL", "UPDRS2.TOTAL", "UPDRS3.TOTAL", "UPDRS4.TOTAL", "UPDRS.TOTAL"]].head())

## ⚙️ Reglas de integridad y prorrateo

**Referencia:**  
Goetz CG, Luo S, Wang L, Tilley BC, LaPelle NR, Stebbins GT. *Handling missing values in the MDS-UPDRS.*  
Mov Disord. 2015 Oct;30(12):1632-8. doi: [10.1002/mds.26153](https://doi.org/10.1002/mds.26153).  
PMID: 25649812; PMCID: PMC5072275.

---

### 🧾 Criterios

Según Goetz et al. (2015), se permite el cálculo prorrateado de la MDS-UPDRS **si no se supera el umbral máximo de ítems faltantes** por parte:

| Parte | Umbral máximo de ítems faltantes | Comentario |
|:------|:--------------------------------:|:------------|
| I     | ≤ 1 ítem                         | Aceptable |
| II    | ≤ 2 ítems (aleatorios) o 1 (fijo) | |
| III   | ≤ 7 ítems (aleatorios) o 3 (fijos) | |
| IV    | 0 ítems                           | Ninguno permitido |

*Se permite: Parte I ≤1 ítem, Parte II ≤2 (aleatorios) o 1 (fijos), Parte III ≤7 (aleatorios) o 3 (fijos), Parte IV ningún ítem; por encima, descartar o imputar con métodos más robustos.*

In [ ]:
# --- 1.1 Umbrales máximos de ítems faltantes (Goetz 2015)
MISSING_RULES = {"I": 1, "II": 2, "III": 7, "IV": 0}

# --- 1.2 Diccionario de ítems por parte (ya los definiste antes)
parte_items = {
    "I": updrs1_cols,
    "II": updrs2_cols,
    "III": updrs3_cols,
    "IV": updrs4_cols,
}

# --- 1.3 Funciones utilitarias
def es_valido_fila(row, parte):
    max_missing = MISSING_RULES[parte]
    n_missing = row[parte_items[parte]].isna().sum()
    return n_missing <= max_missing

def prorratear_fila(row, parte):
    items = parte_items[parte]
    miss = row[items].isna()
    if miss.any():
        factor = len(items) / (~miss).sum()
        row[items] = row[items].fillna(0) * factor
    return row


## 🧮 Cálculo de totales con prorrateo (sin descartar filas)

Este bloque ejecuta las funciones anteriores para:
1. Validar el umbral de faltantes por parte.  
2. Aplicar prorrateo cuando corresponde.  
3. Calcular los totales por parte y el puntaje global.

Notas:

Este procedimiento no descarta filas, pero sólo aplica el prorrateo cuando el número de ítems faltantes está dentro del umbral permitido.

Los totales se recalculan después de imputar los valores proporcionalmente.

El resultado df_clean conserva la misma estructura del DataFrame original con nuevas columnas:

MDS_UPDRS_I, MDS_UPDRS_II, MDS_UPDRS_III, MDS_UPDRS_IV

MDS_UPDRS_TOTAL

In [ ]:
def calcular_totales_con_prorrateo(df):
    df_out = df.copy()
    for parte in ["I", "II", "III", "IV"]:
        # 2.1 Validar umbral de missing
        df_out = df_out[df_out.apply(es_valido_fila, axis=1, args=(parte,))]
        # 2.2 Prorratear si procede
        df_out = df_out.apply(prorratear_fila, axis=1, args=(parte,))
        # 2.3 Sumar puntaje
        df_out[f"MDS_UPDRS_{parte}"] = df_out[parte_items[parte]].sum(axis=1)
    # 2.4 Puntaje total
    df_out["MDS_UPDRS_TOTAL"] = df_out[[f"MDS_UPDRS_{p}" for p in "I II III IV".split()]].sum(axis=1)
    return df_out

# === Ejecuta ===
df_clean = calcular_totales_con_prorrateo(df)


## 🧩 MDS-UPDRS Parte I

Esta sección valida los ítems correspondientes a la **Parte I** de la escala MDS-UPDRS (*Experiencias no motoras de la vida diaria*).  
El objetivo es garantizar que los datos sean numéricos, estén dentro del rango permitido (0–4) y que cada registro contenga el número esperado de ítems válidos (13 para esta parte).

---

### Código para validar MDS-UPDRS Parte I

Notas

Cada ítem de la Parte I debe tener un valor de 0 a 4 (0 = normal, 4 = muy severo).

Si alguna celda tiene valores fuera de rango o vacíos, se marca como no válida.

La columna UPDRS1.completo confirma que la fila contiene los 13 ítems válidos esperados.

UPDRS1.con_errores = True indica que la observación no cumple con los criterios mínimos de integridad.

Este mismo procedimiento puede adaptarse fácilmente a las Partes II–IV modificando el prefijo (UPDRS2., UPDRS3., etc.).

In [ ]:
# Identificar columnas de Parte I
updrs1_cols = [col for col in df.columns if col.startswith("UPDRS1.")]
updrs1_cols = sorted(updrs1_cols)  # Ordenar por si están desordenadas

# Asegurar que las columnas sean numéricas
df[updrs1_cols] = df[updrs1_cols].apply(pd.to_numeric, errors="coerce")

# Validar valor individual entre 0 y 4
for col in updrs1_cols:
    validation_results_updrs[f"{col}.valido"] = df[col].isin([0, 1, 2, 3, 4])

# Validar que haya exactamente 13 ítems válidos
validation_results_updrs["UPDRS1.completo"] = df[updrs1_cols].apply(
    lambda row: row.dropna().isin([0, 1, 2, 3, 4]).sum() == 13, axis=1
)

# Agregar columna de error general para la Parte I
validation_results_updrs["UPDRS1.con_errores"] = ~validation_results_updrs[
    [f"{col}.valido" for col in updrs1_cols] + ["UPDRS1.completo"]
].all(axis=1)

# Mostrar filas con errores
errores_updrs1 = validation_results_updrs[validation_results_updrs["UPDRS1.con_errores"]]

# Visualizar errores
print("Errores detectados en UPDRS Parte I:")
display(errores_updrs1.head(10))  # O usa .to_excel si deseas exportar


## Porcentaje de valores faltantes por ítem — MDS-UPDRS Parte I

Este análisis estima el porcentaje de celdas faltantes por ítem en la Parte I, útil para diagnóstico de calidad de datos y decisiones de imputación.

In [ ]:
# Identificar columnas de la Parte I
updrs1_cols = [col for col in df.columns if col.startswith("UPDRS1.")]
updrs1_cols = sorted(updrs1_cols)

# Calcular porcentaje de missing por ítem
missing_percent_updrs1 = df[updrs1_cols].isna().mean().round(3) * 100

# Convertir a DataFrame para visualizar o exportar
missing_percent_updrs1_df = missing_percent_updrs1.reset_index()
missing_percent_updrs1_df.columns = ["Ítem", "% Missing"]

# Mostrar
print(missing_percent_updrs1_df)



## 🧪 Test de Little — MCAR (Missing Completely at Random)

El **Test de Little MCAR** evalúa si los valores faltantes en los ítems ocurren de manera completamente aleatoria (*Missing Completely at Random*).  
Esto permite decidir si es apropiado aplicar imputación simple o si los datos presentan sesgos sistemáticos.

### 📊 Interpretación de resultados
- Si **p-value < 0.05** → Hay diferencia significativa → **No MCAR** (faltantes no aleatorios).  
- Si **p-value ≥ 0.05** → No hay diferencia significativa → **Posiblemente MCAR** (faltantes aleatorios).

In [ ]:
from scipy.stats import ttest_ind
import pandas as pd

# Asegura que los ítems están en formato numérico
updrs1_cols = sorted([col for col in df.columns if col.startswith("UPDRS1.") and col != "UPDRS1.TOTAL"])
df[updrs1_cols] = df[updrs1_cols].apply(pd.to_numeric, errors='coerce')

# Crear lista para resultados
ttest_results = []

# Comparar cada ítem con los demás
for target_col in updrs1_cols:
    row_results = {"item": target_col}
    for other_col in updrs1_cols:
        if other_col != target_col:
            group1 = df[df[other_col].isnull()][target_col].dropna()
            group2 = df[df[other_col].notnull()][target_col].dropna()

            # Asegurar tamaño mínimo para aplicar prueba t
            if len(group1) >= 5 and len(group2) >= 5:
                stat, p = ttest_ind(group1, group2, equal_var=False, nan_policy='omit')
                row_results[f"p_vs_{other_col}"] = round(p, 4)
            else:
                row_results[f"p_vs_{other_col}"] = None
    ttest_results.append(row_results)

# Convertir a DataFrame y mostrar
ttest_df = pd.DataFrame(ttest_results)
ttest_df



## 🧮 Modelo predictivo del patrón de *missing*

Este modelo evalúa si los valores faltantes pueden **predecirse a partir de otras variables**.  
Se basa en una regresión logística binaria para cada ítem, donde la variable dependiente es “faltante o no faltante”.


### 📊 Interpretación de resultados
- **Pseudo R² bajo (< 0.05)** → la variable objetivo (missing o no) **no se predice bien** → evidencia de **MCAR** (faltantes completamente aleatorios).  
- **Pseudo R² moderado o alto (> 0.10)** → el patrón de faltantes puede estar **relacionado con otras variables** → indicio de **MAR** o **MNAR** (faltantes no completamente aleatorios).

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

predictive_missing_results = []

for target_col in updrs1_cols:
    y = df[target_col].isnull().astype(int)  # variable objetivo: 1 si falta, 0 si no
    predictors = [col for col in updrs1_cols if col != target_col and col != "UPDRS1.VALIDO"]

    X = df[predictors].copy()
    X = X.apply(pd.to_numeric, errors='coerce')

    model_data = pd.concat([X, y], axis=1).dropna()

    if model_data.shape[0] >= 30:
        X_clean = model_data[predictors].astype(float)
        y_clean = model_data[target_col].astype(float)

        X_clean = sm.add_constant(X_clean)

        try:
            model = sm.Logit(y_clean, X_clean).fit(disp=0)
            pseudo_r2 = model.prsquared
            predictive_missing_results.append({
                "Item": target_col,
                "Pseudo R²": round(pseudo_r2, 4),
                "n_obs": model_data.shape[0]
            })
        except Exception as e:
            predictive_missing_results.append({
                "Item": target_col,
                "Pseudo R²": "Error",
                "n_obs": model_data.shape[0],
                "error": str(e)
            })

# Mostrar resultados
predictive_df = pd.DataFrame(predictive_missing_results)
display(predictive_df)


## Imputabilidad por ítem (criterios y reporte)

Se evalúa, para cada ítem de la Parte I, si cumple criterios mínimos para ser imputado:

1) Es numérico.  
2) Sus valores observados están en el rango 0–4.  
3) Porcentaje de valores faltantes por debajo de un umbral (por defecto, 20 %).

El resultado es una tabla con el estado de imputabilidad por ítem.

### Notas
- Ajusta `umbral_missing` según el plan analítico (por ejemplo, 10 %, 15 %, 20 %).  
- Si `Numérico` es falso, convierte previamente con `pd.to_numeric(..., errors="coerce")`.  
- Revisa ítems con `% Missing` alto aunque sean imputables; podrían requerir inspección manual.

In [ ]:
# Definir umbral de imputabilidad (porcentaje de missing aceptable)
umbral_missing = 20

# Revisar si cada ítem está en escala 0–4 y tiene % de missing < 20
imputabilidad = []
for col in updrs1_cols:
    es_numerico = pd.api.types.is_numeric_dtype(df[col])
    en_rango = df[col].dropna().isin([0, 1, 2, 3, 4]).all()
    missing_pct = df[col].isna().mean() * 100
    imputable = es_numerico and en_rango and (missing_pct < umbral_missing)
    imputabilidad.append({
        "Ítem": col,
        "Imputable": imputable,
        "% Missing": round(missing_pct, 2),
        "Numérico": es_numerico,
        "Rango 0–4": en_rango
    })

# Convertir a DataFrame y mostrar
imputabilidad_df = pd.DataFrame(imputabilidad)
print(imputabilidad_df)


## 🤖 Criterios y código para sugerencia de método de imputación

Este bloque genera una **recomendación automática del método de imputación** más adecuado para cada ítem según su porcentaje de valores faltantes y el tipo de variable.

---

### 📊 Criterios de decisión

| % Missing | Tipo / Escala | Método sugerido |
|------------|----------------|-----------------|
| > 30 %     | — | **No imputar** (demasiada pérdida de información) |
| < 5 %      | Ordinal (0–4) | **Moda** |
| 5–20 %     | Ordinal (0–4) | **Mediana por grupo** o **Regresión ordinal** |
| 20–30 %    | Ordinal (0–4) | **KNN Imputation** (para mantener estructura multivariada) |
| —          | Otros tipos | **Evaluar manualmente** |

Notas

La lógica está pensada para ítems ordinales con escala 0–4, como los de la MDS-UPDRS.

Para ítems continuos o binarios, puede adaptarse el bloque tipo dentro de la función ("continuo", "binario").

Si un ítem tiene >30 % de valores faltantes, no debe imputarse: lo recomendable es excluirlo o analizar causas de ausencia.

Los métodos “Mediana por grupo” o “Regresión ordinal” preservan la estructura ordinal del ítem y evitan sesgos sistemáticos.

El método KNN puede ser útil cuando hay correlación fuerte entre ítems de la misma parte.

In [ ]:
# Definir función de recomendación
def sugerir_metodo_imputacion(missing_pct, tipo="ordinal", escala=(0, 4)):
    if missing_pct > 30:
        return "No imputar (>30% missing)"
    elif tipo == "ordinal" and escala == (0, 4):
        if missing_pct < 5:
            return "Moda"
        elif missing_pct < 20:
            return "Mediana por grupo o regresión ordinal"
        else:
            return "KNN imputación"
    else:
        return "Evaluar manualmente"

# Aplicar recomendaciones por ítem
sugerencias = []
for col in updrs1_cols:
    pct_missing = df[col].isna().mean() * 100
    metodo = sugerir_metodo_imputacion(pct_missing)
    sugerencias.append({"Ítem": col, "% Missing": round(pct_missing, 2), "Método sugerido": metodo})

# Convertir a DataFrame y mostrar
sugerencias_df = pd.DataFrame(sugerencias)
print(sugerencias_df)


## 🧩 MDS-UPDRS Parte II

Esta sección replica el proceso de validación aplicado a la Parte I, ahora sobre la **Parte II** (*Experiencias motoras de la vida diaria*).  
Se evalúa integridad, consistencia y completitud de los ítems correspondientes.


In [ ]:
# Identificar columnas de Parte II
updrs2_cols = [col for col in df.columns if col.startswith("UPDRS2.")]
updrs2_cols = sorted(updrs2_cols)  # Ordenar por si están desordenadas

# Asegurar que las columnas sean numéricas
df[updrs2_cols] = df[updrs2_cols].apply(pd.to_numeric, errors="coerce")

# Validar valor individual entre 0 y 4
for col in updrs2_cols:
    validation_results_updrs[f"{col}.valido"] = df[col].isin([0, 1, 2, 3, 4])

# Validar que haya exactamente 13 ítems válidos
validation_results_updrs["UPDRS2.completo"] = df[updrs2_cols].apply(
    lambda row: row.dropna().isin([0, 1, 2, 3, 4]).sum() == 13, axis=1
)

# Agregar columna de error general para la Parte II
validation_results_updrs["UPDRS2.con_errores"] = ~validation_results_updrs[
    [f"{col}.valido" for col in updrs2_cols] + ["UPDRS2.completo"]
].all(axis=1)

# Mostrar filas con errores
errores_updrs2 = validation_results_updrs[validation_results_updrs["UPDRS2.con_errores"]]

# Visualizar errores
print("Errores detectados en UPDRS Parte II:")
display(errores_updrs2.head(10))  # O usa .to_excel si deseas exportar


## Porcentaje de valores faltantes por ítem — MDS-UPDRS Parte II
Evalúa el porcentaje de valores ausentes por cada ítem, detectando posibles áreas problemáticas antes de imputación.


In [ ]:
updrs2_cols = sorted([col for col in df.columns if col.startswith("UPDRS2.")])
missing_percent_updrs2 = df[updrs2_cols].isna().mean().round(3) * 100
missing_percent_updrs2_df = missing_percent_updrs2.reset_index()
missing_percent_updrs2_df.columns = ["Ítem", "% Missing"]
print("Parte II:")
print(missing_percent_updrs2_df)

## 🧪 Test de Little — MCAR (Parte II)
Verifica si los valores faltantes en la Parte II ocurren de forma completamente aleatoria.  
Si *p* ≥ 0.05 → posiblemente MCAR; si *p* < 0.05 → evidencia de no MCAR.


In [ ]:
from scipy.stats import ttest_ind
import pandas as pd

# Asegurar que todas las columnas relevantes estén en formato numérico
def run_ttests_for_part(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

    ttest_results = []

    for target_col in cols:
        row_results = {"item": target_col}
        for other_col in cols:
            if other_col != target_col:
                group1 = df[df[other_col].isnull()][target_col].dropna()
                group2 = df[df[other_col].notnull()][target_col].dropna()

                if len(group1) >= 5 and len(group2) >= 5:
                    stat, p = ttest_ind(group1, group2, equal_var=False, nan_policy='omit')
                    row_results[f"p_vs_{other_col}"] = round(p, 4)
                else:
                    row_results[f"p_vs_{other_col}"] = None
        ttest_results.append(row_results)

    return pd.DataFrame(ttest_results)

# Aplicar a cada parte
ttest_df_part2 = run_ttests_for_part(df, "UPDRS2.")

# Mostrar resultados
print("T-test resultados Parte II:")
display(ttest_df_part2)



## 🧮 Modelo predictivo del patrón de missing — MDS-UPDRS Parte II
Se utiliza regresión logística para estimar si la presencia de datos faltantes en un ítem puede predecirse por otros,  
identificando patrones MAR/MNAR (dependencia estructurada entre variables).


Interpretación del output Pseudo R² bajo (< 0.05): la variable objetivo (missing o no) no se puede predecir bien → evidencia de MCAR.

Pseudo R² moderado/alto (> 0.1): el patrón de missing puede estar relacionado con otras variables → MAR o MNAR.

In [ ]:
import statsmodels.api as sm
import pandas as pd

def run_predictive_missingness(df, prefix):
    # Identificar columnas que comienzan con el prefijo y no son totales
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])

    # Asegurar que sean numéricos
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

    results = []

    for target_col in cols:
        y = df[target_col].isnull().astype(int)
        predictors = [col for col in cols if col != target_col]

        X = df[predictors].copy()
        X = X.apply(pd.to_numeric, errors='coerce')

        data = pd.concat([X, y], axis=1).dropna()

        if data.shape[0] >= 30:
            X_clean = data[predictors].astype(float)
            y_clean = data[target_col].astype(float)

            X_clean = sm.add_constant(X_clean)

            try:
                model = sm.Logit(y_clean, X_clean).fit(disp=0)
                pseudo_r2 = model.prsquared
                results.append({
                    "Item": target_col,
                    "Pseudo R²": round(pseudo_r2, 4),
                    "n_obs": data.shape[0]
                })
            except Exception as e:
                results.append({
                    "Item": target_col,
                    "Pseudo R²": "Error",
                    "n_obs": data.shape[0],
                    "error": str(e)
                })

    return pd.DataFrame(results)

# Ejecutar para cada parte
predictive_df_part2 = run_predictive_missingness(df, "UPDRS2.")


# Visualizar
print("Parte II:")
print(predictive_df_part2)



## ⚙️ Imputabilidad de ítems — Parte II
Evalúa si cada ítem cumple criterios mínimos de imputabilidad (tipo numérico, rango 0-4, < 20 % missing).  
El resultado clasifica cada variable como *imputable* o *no imputable*.


In [ ]:
# Umbral de % de missing aceptable
umbral_missing = 20

def evaluar_imputabilidad(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

    imputabilidad = []
    for col in cols:
        es_numerico = pd.api.types.is_numeric_dtype(df[col])
        en_rango = df[col].dropna().isin([0, 1, 2, 3, 4]).all()
        missing_pct = df[col].isna().mean() * 100
        imputable = es_numerico and en_rango and (missing_pct < umbral_missing)
        imputabilidad.append({
            "Ítem": col,
            "Imputable": imputable,
            "% Missing": round(missing_pct, 2),
            "Numérico": es_numerico,
            "Rango 0–4": en_rango
        })

    return pd.DataFrame(imputabilidad)

# Ejecutar para cada parte
imputabilidad_part2 = evaluar_imputabilidad(df, "UPDRS2.")


# Mostrar
print("Parte II:")
print(imputabilidad_part2)




## 🤖 Sugerencia de método de imputación — Parte II
Propone el método más adecuado (Moda, Mediana por grupo, KNN, etc.) según el porcentaje de celdas faltantes.  
Permite estandarizar el tratamiento de datos ausentes de la Parte II.


In [ ]:
import pandas as pd

# Función de recomendación de imputación
def sugerir_metodo_imputacion(missing_pct, tipo="ordinal", escala=(0, 4)):
    if missing_pct > 30:
        return "No imputar (>30% missing)"
    elif tipo == "ordinal" and escala == (0, 4):
        if missing_pct < 5:
            return "Moda"
        elif missing_pct < 20:
            return "Mediana por grupo o regresión ordinal"
        else:
            return "KNN imputación"
    else:
        return "Evaluar manualmente"

# Función para aplicar a cada parte
def generar_sugerencias(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    sugerencias = []
    for col in cols:
        pct_missing = df[col].isna().mean() * 100
        metodo = sugerir_metodo_imputacion(pct_missing)
        sugerencias.append({
            "Ítem": col,
            "% Missing": round(pct_missing, 2),
            "Método sugerido": metodo
        })
    return pd.DataFrame(sugerencias)

# Aplicar y mostrar resultados
sugerencias_updrs2 = generar_sugerencias(df, "UPDRS2.")

print("Sugerencias Parte II:")
print(sugerencias_updrs2.to_string(index=False))



## 🔢 MDS-UPDRS Parte III

Esta sección aborda la **Parte III (Exploración motora)**, con más ítems y mayor riesgo de valores ausentes.  
Se aplican los mismos procedimientos: validación, análisis de faltantes, MCAR test, modelo predictivo, imputabilidad y sugerencia de método.


In [ ]:
# Identificar columnas de Parte III
updrs3_cols = [col for col in df.columns if col.startswith("UPDRS3.")]
updrs3_cols = sorted(updrs3_cols)  # Ordenar por si están desordenadas

# Asegurar que las columnas sean numéricas
df[updrs3_cols] = df[updrs3_cols].apply(pd.to_numeric, errors="coerce")

# Validar valor individual entre 0 y 4
for col in updrs3_cols:
    validation_results_updrs[f"{col}.valido"] = df[col].isin([0, 1, 2, 3, 4])

# Validar que haya exactamente 33 ítems válidos
validation_results_updrs["UPDRS3.completo"] = df[updrs3_cols].apply(
    lambda row: row.dropna().isin([0, 1, 2, 3, 4]).sum() == 33, axis=1
)

# Agregar columna de error general para la Parte III
validation_results_updrs["UPDRS3.con_errores"] = ~validation_results_updrs[
    [f"{col}.valido" for col in updrs3_cols] + ["UPDRS3.completo"]
].all(axis=1)

# Mostrar filas con errores
errores_updrs3 = validation_results_updrs[validation_results_updrs["UPDRS3.con_errores"]]

# Visualizar errores
print("Errores detectados en UPDRS Parte III:")
display(errores_updrs3.head(10))  # O usa .to_excel si deseas exportar


## Porcentaje de valores faltantes por ítem — MDS-UPDRS Parte III
Evalúa el porcentaje de valores ausentes por cada ítem, detectando posibles áreas problemáticas antes de imputación.


In [ ]:
updrs3_cols = sorted([col for col in df.columns if col.startswith("UPDRS3.")])
missing_percent_updrs3 = df[updrs3_cols].isna().mean().round(3) * 100
missing_percent_updrs3_df = missing_percent_updrs3.reset_index()
missing_percent_updrs3_df.columns = ["Ítem", "% Missing"]
print("\nParte III:")
print(missing_percent_updrs3_df)

## 🧪 Test de Little — MCAR (Parte III)
Verifica si los valores faltantes en la Parte II ocurren de forma completamente aleatoria.  
Si *p* ≥ 0.05 → posiblemente MCAR; si *p* < 0.05 → evidencia de no MCAR.



In [ ]:
from scipy.stats import ttest_ind
import pandas as pd

# Asegurar que todas las columnas relevantes estén en formato numérico
def run_ttests_for_part(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

    ttest_results = []

    for target_col in cols:
        row_results = {"item": target_col}
        for other_col in cols:
            if other_col != target_col:
                group1 = df[df[other_col].isnull()][target_col].dropna()
                group2 = df[df[other_col].notnull()][target_col].dropna()

                if len(group1) >= 5 and len(group2) >= 5:
                    stat, p = ttest_ind(group1, group2, equal_var=False, nan_policy='omit')
                    row_results[f"p_vs_{other_col}"] = round(p, 4)
                else:
                    row_results[f"p_vs_{other_col}"] = None
        ttest_results.append(row_results)

    return pd.DataFrame(ttest_results)

# Aplicar a cada parte

ttest_df_part3 = run_ttests_for_part(df, "UPDRS3.")

# Mostrar resultados

print("T-test resultados Parte III:")
display(ttest_df_part3)




## 🧮 Modelo predictivo del patrón de missing — MDS-UPDRS Parte III
Se utiliza regresión logística para estimar si la presencia de datos faltantes en un ítem puede predecirse por otros,  
identificando patrones MAR/MNAR (dependencia estructurada entre variables).


Interpretación del output Pseudo R² bajo (< 0.05): la variable objetivo (missing o no) no se puede predecir bien → evidencia de MCAR.

Pseudo R² moderado/alto (> 0.1): el patrón de missing puede estar relacionado con otras variables → MAR o MNAR.

In [ ]:
import statsmodels.api as sm
import pandas as pd

def run_predictive_missingness(df, prefix):
    # Identificar columnas que comienzan con el prefijo y no son totales
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])

    # Asegurar que sean numéricos
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

    results = []

    for target_col in cols:
        y = df[target_col].isnull().astype(int)
        predictors = [col for col in cols if col != target_col]

        X = df[predictors].copy()
        X = X.apply(pd.to_numeric, errors='coerce')

        data = pd.concat([X, y], axis=1).dropna()

        if data.shape[0] >= 30:
            X_clean = data[predictors].astype(float)
            y_clean = data[target_col].astype(float)

            X_clean = sm.add_constant(X_clean)

            try:
                model = sm.Logit(y_clean, X_clean).fit(disp=0)
                pseudo_r2 = model.prsquared
                results.append({
                    "Item": target_col,
                    "Pseudo R²": round(pseudo_r2, 4),
                    "n_obs": data.shape[0]
                })
            except Exception as e:
                results.append({
                    "Item": target_col,
                    "Pseudo R²": "Error",
                    "n_obs": data.shape[0],
                    "error": str(e)
                })

    return pd.DataFrame(results)

# Ejecutar para cada parte

predictive_df_part3 = run_predictive_missingness(df, "UPDRS3.")


# Visualizar

print("\nParte III:")
print(predictive_df_part3)



## ⚙️ Imputabilidad de ítems — Parte III
Evalúa si cada ítem cumple criterios mínimos de imputabilidad (tipo numérico, rango 0-4, < 20 % missing).  
El resultado clasifica cada variable como *imputable* o *no imputable*.


In [ ]:
# Umbral de % de missing aceptable
umbral_missing = 20

def evaluar_imputabilidad(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

    imputabilidad = []
    for col in cols:
        es_numerico = pd.api.types.is_numeric_dtype(df[col])
        en_rango = df[col].dropna().isin([0, 1, 2, 3, 4]).all()
        missing_pct = df[col].isna().mean() * 100
        imputable = es_numerico and en_rango and (missing_pct < umbral_missing)
        imputabilidad.append({
            "Ítem": col,
            "Imputable": imputable,
            "% Missing": round(missing_pct, 2),
            "Numérico": es_numerico,
            "Rango 0–4": en_rango
        })

    return pd.DataFrame(imputabilidad)

# Ejecutar para cada parte

imputabilidad_part3 = evaluar_imputabilidad(df, "UPDRS3.")


# Mostrar

print("\nParte III:")
print(imputabilidad_part3)



## 🤖 Sugerencia de método de imputación — Parte III
Propone el método más adecuado (Moda, Mediana por grupo, KNN, etc.) según el porcentaje de celdas faltantes.  
Permite estandarizar el tratamiento de datos ausentes de la Parte II.


In [ ]:
import pandas as pd

# Función de recomendación de imputación
def sugerir_metodo_imputacion(missing_pct, tipo="ordinal", escala=(0, 4)):
    if missing_pct > 30:
        return "No imputar (>30% missing)"
    elif tipo == "ordinal" and escala == (0, 4):
        if missing_pct < 5:
            return "Moda"
        elif missing_pct < 20:
            return "Mediana por grupo o regresión ordinal"
        else:
            return "KNN imputación"
    else:
        return "Evaluar manualmente"

# Función para aplicar a cada parte
def generar_sugerencias(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    sugerencias = []
    for col in cols:
        pct_missing = df[col].isna().mean() * 100
        metodo = sugerir_metodo_imputacion(pct_missing)
        sugerencias.append({
            "Ítem": col,
            "% Missing": round(pct_missing, 2),
            "Método sugerido": metodo
        })
    return pd.DataFrame(sugerencias)

# Aplicar y mostrar resultados
sugerencias_updrs3 = generar_sugerencias(df, "UPDRS3.")


print("\nSugerencias Parte III:")
print(sugerencias_updrs3.to_string(index=False))



## ⚖️ MDS-UPDRS Parte IV

La **Parte IV (Complicaciones motoras)** tiene menos ítems, por lo que se espera menor proporción de valores faltantes.  
Se aplican los mismos pasos de verificación e imputación utilizados en las Partes I–III para garantizar consistencia del dataset completo.


In [ ]:
# Identificar columnas de Parte IV
updrs4_cols = [col for col in df.columns if col.startswith("UPDRS4.")]
updrs4_cols = sorted(updrs4_cols)  # Ordenar por si están desordenadas

# Asegurar que las columnas sean numéricas
df[updrs4_cols] = df[updrs4_cols].apply(pd.to_numeric, errors="coerce")

# Validar valor individual entre 0 y 4
for col in updrs4_cols:
    validation_results_updrs[f"{col}.valido"] = df[col].isin([0, 1, 2, 3, 4])

# Validar que haya exactamente 6 ítems válidos
validation_results_updrs["UPDRS4.completo"] = df[updrs4_cols].apply(
    lambda row: row.dropna().isin([0, 1, 2, 3, 4]).sum() == 6, axis=1
)

# Agregar columna de error general para la Parte IV
validation_results_updrs["UPDRS4.con_errores"] = ~validation_results_updrs[
    [f"{col}.valido" for col in updrs4_cols] + ["UPDRS4.completo"]
].all(axis=1)

# Mostrar filas con errores
errores_updrs4 = validation_results_updrs[validation_results_updrs["UPDRS4.con_errores"]]

# Visualizar errores
print("Errores detectados en UPDRS Parte IV:")
display(errores_updrs4.head(10))  # O usa .to_excel si deseas exportar


## Porcentaje de valores faltantes por ítem — MDS-UPDRS Parte IV
Evalúa el porcentaje de valores ausentes por cada ítem, detectando posibles áreas problemáticas antes de imputación.


In [ ]:
updrs4_cols = sorted([col for col in df.columns if col.startswith("UPDRS4.")])
missing_percent_updrs4 = df[updrs4_cols].isna().mean().round(3) * 100
missing_percent_updrs4_df = missing_percent_updrs4.reset_index()
missing_percent_updrs4_df.columns = ["Ítem", "% Missing"]
print("\nParte IV:")
print(missing_percent_updrs4_df)

## 🧪 Test de Little — MCAR (Parte IV)
Verifica si los valores faltantes en la Parte II ocurren de forma completamente aleatoria.  
Si *p* ≥ 0.05 → posiblemente MCAR; si *p* < 0.05 → evidencia de no MCAR.

Si el p-value < 0.05, hay diferencia significativa → No MCAR

Si el p-value ≥ 0.05, no hay diferencia → Posiblemente MCAR

In [ ]:
from scipy.stats import ttest_ind
import pandas as pd

# Asegurar que todas las columnas relevantes estén en formato numérico
def run_ttests_for_part(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

    ttest_results = []

    for target_col in cols:
        row_results = {"item": target_col}
        for other_col in cols:
            if other_col != target_col:
                group1 = df[df[other_col].isnull()][target_col].dropna()
                group2 = df[df[other_col].notnull()][target_col].dropna()

                if len(group1) >= 5 and len(group2) >= 5:
                    stat, p = ttest_ind(group1, group2, equal_var=False, nan_policy='omit')
                    row_results[f"p_vs_{other_col}"] = round(p, 4)
                else:
                    row_results[f"p_vs_{other_col}"] = None
        ttest_results.append(row_results)

    return pd.DataFrame(ttest_results)

# Aplicar a cada parte

ttest_df_part4 = run_ttests_for_part(df, "UPDRS4.")

# Mostrar resultados


print("T-test resultados Parte IV:")
display(ttest_df_part4)


## 🧮 Modelo predictivo del patrón de missing — MDS-UPDRS Parte IV
Se utiliza regresión logística para estimar si la presencia de datos faltantes en un ítem puede predecirse por otros,  
identificando patrones MAR/MNAR (dependencia estructurada entre variables).


Interpretación del output Pseudo R² bajo (< 0.05): la variable objetivo (missing o no) no se puede predecir bien → evidencia de MCAR.

Pseudo R² moderado/alto (> 0.1): el patrón de missing puede estar relacionado con otras variables → MAR o MNAR.

In [ ]:
import statsmodels.api as sm
import pandas as pd

def run_predictive_missingness(df, prefix):
    # Identificar columnas que comienzan con el prefijo y no son totales
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])

    # Asegurar que sean numéricos
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

    results = []

    for target_col in cols:
        y = df[target_col].isnull().astype(int)
        predictors = [col for col in cols if col != target_col]

        X = df[predictors].copy()
        X = X.apply(pd.to_numeric, errors='coerce')

        data = pd.concat([X, y], axis=1).dropna()

        if data.shape[0] >= 30:
            X_clean = data[predictors].astype(float)
            y_clean = data[target_col].astype(float)

            X_clean = sm.add_constant(X_clean)

            try:
                model = sm.Logit(y_clean, X_clean).fit(disp=0)
                pseudo_r2 = model.prsquared
                results.append({
                    "Item": target_col,
                    "Pseudo R²": round(pseudo_r2, 4),
                    "n_obs": data.shape[0]
                })
            except Exception as e:
                results.append({
                    "Item": target_col,
                    "Pseudo R²": "Error",
                    "n_obs": data.shape[0],
                    "error": str(e)
                })

    return pd.DataFrame(results)

# Ejecutar para cada parte
predictive_df_part4 = run_predictive_missingness(df, "UPDRS4.")

# Visualizar

print("\nParte IV:")
print(predictive_df_part4)


## ⚙️ Imputabilidad de ítems — Parte IV
Evalúa si cada ítem cumple criterios mínimos de imputabilidad (tipo numérico, rango 0-4, < 20 % missing).  
El resultado clasifica cada variable como *imputable* o *no imputable*.


In [ ]:
# Umbral de % de missing aceptable
umbral_missing = 20

def evaluar_imputabilidad(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

    imputabilidad = []
    for col in cols:
        es_numerico = pd.api.types.is_numeric_dtype(df[col])
        en_rango = df[col].dropna().isin([0, 1, 2, 3, 4]).all()
        missing_pct = df[col].isna().mean() * 100
        imputable = es_numerico and en_rango and (missing_pct < umbral_missing)
        imputabilidad.append({
            "Ítem": col,
            "Imputable": imputable,
            "% Missing": round(missing_pct, 2),
            "Numérico": es_numerico,
            "Rango 0–4": en_rango
        })

    return pd.DataFrame(imputabilidad)

# Ejecutar para cada parte

imputabilidad_part4 = evaluar_imputabilidad(df, "UPDRS4.")

# Mostrar

print("\nParte IV:")
print(imputabilidad_part4)


## 🤖 Sugerencia de método de imputación — Parte IV
Propone el método más adecuado (Moda, Mediana por grupo, KNN, etc.) según el porcentaje de celdas faltantes.  
Permite estandarizar el tratamiento de datos ausentes de la Parte IV


In [ ]:
import pandas as pd

# Función de recomendación de imputación
def sugerir_metodo_imputacion(missing_pct, tipo="ordinal", escala=(0, 4)):
    if missing_pct > 30:
        return "No imputar (>30% missing)"
    elif tipo == "ordinal" and escala == (0, 4):
        if missing_pct < 5:
            return "Moda"
        elif missing_pct < 20:
            return "Mediana por grupo o regresión ordinal"
        else:
            return "KNN imputación"
    else:
        return "Evaluar manualmente"

# Función para aplicar a cada parte
def generar_sugerencias(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    sugerencias = []
    for col in cols:
        pct_missing = df[col].isna().mean() * 100
        metodo = sugerir_metodo_imputacion(pct_missing)
        sugerencias.append({
            "Ítem": col,
            "% Missing": round(pct_missing, 2),
            "Método sugerido": metodo
        })
    return pd.DataFrame(sugerencias)

# Aplicar y mostrar resultados

sugerencias_updrs4 = generar_sugerencias(df, "UPDRS4.")


print("\nSugerencias Parte IV:")
print(sugerencias_updrs4.to_string(index=False))



## 📋 Resumen de sugerencias de imputación

En esta sección se consolidan las recomendaciones de imputación para **todas las partes de la MDS-UPDRS (I–IV)**.  
El objetivo es automatizar la selección de estrategias según el tipo de ítem, su rango (0–4) y el porcentaje de datos faltantes.

Cada parte genera su propio resumen con los siguientes campos:

- **Ítem:** nombre de la variable.  
- **% Missing:** proporción de valores faltantes.  
- **Método sugerido:** técnica recomendada para imputar (o no imputar) según criterios definidos.

---


In [ ]:
import pandas as pd

# Función de recomendación
def sugerir_metodo_imputacion(missing_pct, tipo="ordinal", escala=(0, 4)):
    if missing_pct > 30:
        return "No imputar (>30% missing)"
    elif tipo == "ordinal" and escala == (0, 4):
        if missing_pct < 5:
            return "Moda"
        elif missing_pct < 20:
            return "Mediana por grupo o regresión ordinal"
        else:
            return "KNN imputación"
    else:
        return "Evaluar manualmente"

# Función para generar recomendaciones
def generar_sugerencias(df, prefix):
    cols = sorted([col for col in df.columns if col.startswith(prefix) and not col.endswith("TOTAL")])
    sugerencias = []
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # asegurar tipo numérico
        pct_missing = df[col].isna().mean() * 100
        metodo = sugerir_metodo_imputacion(pct_missing)
        sugerencias.append({
            "Ítem": col,
            "% Missing": round(pct_missing, 2),
            "Método sugerido": metodo
        })
    return pd.DataFrame(sugerencias)

# Aplicar a todas las partes
sugerencias1 = generar_sugerencias(df, "UPDRS1.")
sugerencias2 = generar_sugerencias(df, "UPDRS2.")
sugerencias3 = generar_sugerencias(df, "UPDRS3.")
sugerencias4 = generar_sugerencias(df, "UPDRS4.")

# Función para colorear bonito
def estilo_color(df):
    def highlight_method(val):
        if "No imputar" in val:
            return 'background-color: #ffcccc'
        elif "KNN" in val:
            return 'background-color: #fff2cc'
        elif "Mediana" in val:
            return 'background-color: #ccffcc'
        elif "Moda" in val:
            return 'background-color: #cce5ff'
        else:
            return ''
    return df.style.applymap(highlight_method, subset=["Método sugerido"]).format({"% Missing": "{:.2f}"})

# Mostrar
print("Parte I")
display(estilo_color(sugerencias1))

print("Parte II")
display(estilo_color(sugerencias2))

print("Parte III")
display(estilo_color(sugerencias3))

print("Parte IV")
display(estilo_color(sugerencias4))



## Clasificación de severidad por puntos de corte (triangulación)

Este bloque clasifica la severidad por **parte** de la MDS-UPDRS y una **severidad global** basada en la categoría más alta entre I–IV.  
Los puntos de corte provienen de una triangulación (literatura + distribución empírica). Ajusta los umbrales a tu cohorte si es necesario.

### Puntos de corte propuestos
| Parte | Leve (≤) | Moderada (≤) | Grave (> moderada) |
|------:|---------:|-------------:|--------------------:|
| I     | 10       | 21           |                     |
| II    | 12       | 29           |                     |
| III   | 32       | 58           |                     |
| IV    | 4        | 12           |                     |

> Regla: si el puntaje de la parte ≤ *mild_max* → **leve**; si ≤ *mod_max* → **moderada**; si > *mod_max* → **grave**.


In [ ]:
# --- 1. Definir puntos de corte (triangulación del artículo) ---------------
CUTS = {
    "I":   {"mild_max": 10, "mod_max": 21},
    "II":  {"mild_max": 12, "mod_max": 29},
    "III": {"mild_max": 32, "mod_max": 58},
    "IV":  {"mild_max":  4, "mod_max": 12},
}

def clasificar_parte(score, cut):
    if score <= cut["mild_max"]:
        return "leve"
    elif score <= cut["mod_max"]:
        return "moderada"
    else:
        return "grave"

# --- 2. Etiquetar severidad por parte ---------------------------------------
for parte in ["I", "II", "III", "IV"]:
    col_score = f"MDS_UPDRS_{parte}"
    col_sev   = f"sev_{parte}"
    df_clean[col_sev] = df_clean[col_score].apply(lambda x: clasificar_parte(x, CUTS[parte]))

# --- 3. Severidad global (opcional) -----------------------------------------
# Regla práctica: la categoría más alta entre las cuatro partes
niveles = {"leve": 1, "moderada": 2, "grave": 3}

def severidad_global(row):
    nivel_max = max(row[[f"sev_{p}" for p in ["I","II","III","IV"]]],
                    key=lambda x: niveles[x])
    return nivel_max

df_clean["sev_global"] = df_clean.apply(severidad_global, axis=1)

# --- 4. Vista rápida ---------------------------------------------------------
display(df_clean[["num.consec", "fecha.eval",
                  "MDS_UPDRS_I", "MDS_UPDRS_II", "MDS_UPDRS_III", "MDS_UPDRS_IV",
                  "sev_I", "sev_II", "sev_III", "sev_IV", "sev_global"]].head())


## 📈 Métricas longitudinales: deltas y pendiente anual

Este bloque calcula **variaciones intraindividuales (Δ)** y la **pendiente anualizada** de las puntuaciones MDS-UPDRS  
para evaluar progresión clínica en seguimientos longitudinales.

---

### 🔧 Preprocesamiento
1. **Conversión de fechas:** se transforma `fecha.eval` a formato `datetime`.  
2. **Normalización de columnas numéricas:** todas las columnas `MDS_UPDRS_*` se convierten a tipo numérico (`float`).  
3. **Verificación del total:** si la columna `MDS_UPDRS_TOTAL` no existe, se genera automáticamente como la suma de las partes I–IV.

---

### ⏳ Orden cronológico y cálculo temporal
- Se ordenan las observaciones por sujeto (`num.consec`) y fecha.  
- Se calcula el **tiempo transcurrido en años** desde la primera visita individual mediante:
  \[
  \text{tiempo\_años} = \frac{(\text{fecha.eval} - \text{fecha.inicial})}{365.25}
  \]

---

### 🔁 Deltas intraindividuales
- Se calcula la **variación respecto a la línea base** (primera visita) para cada parte y para el total:  
  \[
  \Delta_{\text{parte}} = \text{UPDRS}_{\text{parte}}(t) - \text{UPDRS}_{\text{parte}}(t_0)
  \]
- Estas métricas permiten estimar la progresión sintomática a lo largo del tiempo.

---

### 🧮 Variables resultantes
| Variable | Descripción |
|-----------|-------------|
| `tiempo_años` | Tiempo desde la primera evaluación (años) |
| `Δ_I`, `Δ_II`, `Δ_III`, `Δ_TOTAL` | Cambios absolutos por parte y global |
| `MDS_UPDRS_TOTAL` | Puntaje total sumado automáticamente si no existía |

---

> **Interpretación:**  
> - Un Δ positivo indica **empeoramiento** en la puntuación.  
> - La pendiente anual puede estimarse aplicando regresión lineal (`Δ / tiempo_años`) o modelos mixtos según la estructura del seguimiento.  
> - Este bloque es fundamental para construir métricas de progresión y modelado longitudinal.



In [ ]:
from datetime import timedelta
import pandas as pd

# --- 0. Asegura tipos --------------------------------------------------------
# 0.1 Fechas
df_clean["fecha.eval"] = pd.to_datetime(df_clean["fecha.eval"], errors="coerce")

# 0.2 Puntajes UPDRS: fuerza a numérico; lo que no sea número se vuelve NaN
score_cols = [c for c in df_clean.columns if c.startswith("MDS_UPDRS_")]
df_clean[score_cols] = df_clean[score_cols].apply(pd.to_numeric, errors="coerce")

# 0.3 Si aún no existe el total, créalo rápido
if "MDS_UPDRS_TOTAL" not in df_clean.columns:
    df_clean["MDS_UPDRS_TOTAL"] = df_clean[[f"MDS_UPDRS_{p}"
                                            for p in ["I", "II", "III", "IV"]]].sum(axis=1)

# --- 1. Orden cronológico ----------------------------------------------------
df_clean = df_clean.sort_values(["num.consec", "fecha.eval"])

# --- 2. Tiempo (años) desde la primera visita individual ---------------------
df_clean["tiempo_años"] = (
    df_clean.groupby("num.consec")["fecha.eval"]
            .transform(lambda x: (x - x.min()).dt.total_seconds() / (365.25 * 24 * 3600))
)

# --- 3. Δ vs. línea base -----------------------------------------------------
for parte in ["I", "II", "III", "TOTAL"]:
    base = df_clean.groupby("num.consec")[f"MDS_UPDRS_{parte}"].transform("first")
    df_clean[f"Δ_{parte}"] = df_clean[f"MDS_UPDRS_{parte}"] - base


## 📊 Visualización rápida de los resultados (`df_clean`)

Este bloque permite revisar de forma inmediata la coherencia general de los datos procesados,  
la distribución de severidad global en la última visita de cada paciente y la tendencia media del puntaje total de MDS-UPDRS en el tiempo.

---

### 1️⃣ Vista rápida de registros procesados
Se inspeccionan las primeras filas relevantes de la base limpia (`df_clean`), mostrando:
- **Identificador del paciente (`num.consec`)**
- **Fecha de evaluación (`fecha.eval`)**
- **Puntaje total (`MDS_UPDRS_TOTAL`)**
- **Delta total respecto a la línea base (`Δ_TOTAL`)**
- **Clasificación de severidad global (`sev_global`)**

> Esta vista sirve para verificar que la estructura de columnas y los cálculos previos se realizaron correctamente.

---

### 2️⃣ Distribución de la severidad global (última visita)
Se obtiene la categoría de **severidad global** de la última visita de cada paciente y se calcula la distribución de frecuencias:
- Permite identificar cuántos casos están en fases **leves**, **moderadas** o **graves**.
- Es útil como control de coherencia y para visualizar el **estado clínico final de la cohorte**.

```python
print("Distribución de severidad global (última visita):")
print(ultima_visita["sev_global"].value_counts())


In [ ]:
# ---------------------------------------------------------------
# VISUALIZACIÓN RÁPIDA DE LOS RESULTADOS EN df_clean
# ---------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt

# 1) Vista rápida de las primeras visitas procesadas
cols_vista = ["num.consec", "fecha.eval",
              "MDS_UPDRS_TOTAL", "Δ_TOTAL", "sev_global"]

if 'df_clean' not in globals():
    raise NameError("No se encontró la variable `df_clean`. "
                    "Ejecuta primero el script de procesamiento.")

display(df_clean[cols_vista].head(10))

# 2) Distribución de la severidad global (última visita por paciente)
ultima_visita = (
    df_clean.sort_values(["num.consec", "fecha.eval"])
            .groupby("num.consec")
            .tail(1)
)
print("\nDistribución de severidad global (última visita):")
print(ultima_visita["sev_global"].value_counts())

# 3) Progresión media del puntaje total a lo largo del tiempo
mean_prog = (
    df_clean.groupby("tiempo_años")["Δ_TOTAL"]
            .mean()
            .sort_index()
)

plt.figure()
plt.plot(mean_prog.index, mean_prog.values, marker='o')
plt.xlabel("Tiempo (años)")
plt.ylabel("Δ MDS-UPDRS Total")
plt.title("Progresión media del MDS-UPDRS Total")
plt.grid(True)
plt.show()


## 🎯 Etiquetas de cambio clínicamente importante (MCID + evento de progresión)

Esta sección implementa las **banderas de cambio clínicamente significativo (MCID)** para identificar  
mejoras o empeoramientos relevantes en las puntuaciones MDS-UPDRS a nivel individual, así como un  
evento de progresión motora (+5 puntos en Parte III, estado OFF).

---

### 🔹 4.1 Umbrales MCID publicados

| Parte | MCID (puntos) | Interpretación |
|:------|:--------------|:---------------|
| I     | ±2.5          | Cambio clínicamente relevante en síntomas no motores |
| II    | ±3.0          | Cambio clínicamente relevante en actividades de la vida diaria |
| III (mejora) | –3.25 | Mejora significativa del desempeño motor |
| III (empeora) | +4.63 | Empeoramiento clínicamente importante del desempeño motor |

> **Referencia:**  
> Horvath et al., *Movement Disorders Clinical Practice*, 2020.  
> Li et al., *Parkinsonism & Related Disorders*, 2021.

---

### 🔸 4.2 Creación de indicadores (flags) MCID

Cada variable de delta (`Δ_I`, `Δ_II`, `Δ_III`) se evalúa contra los umbrales definidos:  
- Si el cambio absoluto excede el MCID → **cambio clínicamente importante**.  
- En la Parte III se generan dos indicadores separados:
  - `mcid_III_mej`: mejora significativa (≤ –3.25).  
  - `mcid_III_emp`: empeoramiento significativo (≥ +4.63).

Ejemplo de cálculo:

```python
df_clean["mcid_I"]        = df_clean["Δ_I"].abs()   >= MCID["I"]
df_clean["mcid_II"]       = df_clean["Δ_II"].abs()  >= MCID["II"]
df_clean["mcid_III_mej"]  = df_clean["Δ_III"] <= -MCID["III_mej"]
df_clean["mcid_III_emp"]  = df_clean["Δ_III"] >=  MCID["III_emp"]


In [ ]:
# 4.1 Umbrales publicados
MCID = {
    "I":  2.5,   # ±2.5 pts
    "II": 3.0,   # ±3.0 pts
    "III_mej": 3.25,
    "III_emp": 4.63
}

# 4.2 Flags MCID
df_clean["mcid_I"]        = df_clean["Δ_I"].abs()  >= MCID["I"]
df_clean["mcid_II"]       = df_clean["Δ_II"].abs() >= MCID["II"]
df_clean["mcid_III_mej"]  = df_clean["Δ_III"] <= -MCID["III_mej"]
df_clean["mcid_III_emp"]  = df_clean["Δ_III"] >=  MCID["III_emp"]

# 4.3 Evento de progresión motora (+5 pts Parte III OFF)
df_clean["event_progIII"] = df_clean["Δ_III"] >= 5


## ⏱️ Pendiente individual y análisis “time-to-event”

Esta sección prepara el entorno para el cálculo de **pendientes de progresión individual** y el análisis de **tiempo hasta evento clínico (time-to-event)**, típicamente mediante modelos de supervivencia de Cox.

---

### 🔧 Instalación de dependencias
Se instala la librería `lifelines`, ampliamente utilizada en análisis de supervivencia (Kaplan-Meier, CoxPH, etc.):



In [ ]:
!pip install -q lifelines


In [ ]:
import statsmodels.formula.api as smf
from lifelines import CoxPHFitter


## 📈 Modelo Mixto Robusto: Pendiente Individual (MDS-UPDRS III)

Esta sección estima la **pendiente individual de progresión motora** (Parte III de la MDS-UPDRS) mediante un **modelo lineal mixto (Mixed-Effects Model)**, que permite capturar tanto la tendencia general como la variabilidad entre pacientes.

---

### ⚙️ 0. Preparación de los datos
1. Se genera una **copia de trabajo** (`df_mlm`) a partir del dataframe limpio (`df_clean`).  
2. Se seleccionan las variables necesarias:
   - `num.consec` → identificador del paciente.  
   - `MDS_UPDRS_III` → puntuación motora.  
   - `tiempo_años` → tiempo desde la línea base.  
3. Se convierten los tipos de datos y se eliminan valores faltantes.

> **Criterio de inclusión:** solo se conservan pacientes con **≥ 2 visitas** y **≥ 2 puntos de tiempo distintos**.

---

### 🧮 1. Especificación del modelo

El modelo lineal mixto ajusta la relación:
\[
\text{MDS\_UPDRS\_III}_{ij} = \beta_0 + \beta_1 \times \text{tiempo\_años}_{ij} + u_{0i} + u_{1i} \times \text{tiempo\_años}_{ij} + \varepsilon_{ij}
\]

Donde:
- \( \beta_0, \beta_1 \) → efectos fijos (intercepto y pendiente media poblacional).  
- \( u_{0i}, u_{1i} \) → efectos aleatorios por paciente (variación interindividual).  
- \( \varepsilon_{ij} \) → error residual intraindividual.

El modelo se implementa así:

```python
modelo = smf.mixedlm(
    formula="MDS_UPDRS_III ~ tiempo_años",
    data=df_mlm,
    groups=df_mlm["num.consec"],
    re_formula="~tiempo_años"  # pendiente aleatoria por paciente
).fit(reml=False, method="lbfgs")


In [ ]:
# --------------------------------------------------------------
#  MIXED-LM ROBUSTO: PENDIENTE INDIVIDUAL (MDS-UPDRS III)
# --------------------------------------------------------------
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# 0. Copia de trabajo
df_mlm = df_clean.copy()

# 1. Conversión de tipos y descarte de NaN
cols_req = ["num.consec", "MDS_UPDRS_III", "tiempo_años"]
df_mlm = df_mlm[cols_req].dropna()

df_mlm["num.consec"]      = df_mlm["num.consec"].astype("category")
df_mlm["MDS_UPDRS_III"]   = pd.to_numeric(df_mlm["MDS_UPDRS_III"], errors="coerce")
df_mlm["tiempo_años"]     = pd.to_numeric(df_mlm["tiempo_años"],  errors="coerce")
df_mlm = df_mlm.dropna()

# 2. Mantener solo pacientes con ≥2 visitas *y* ≥2 tiempos distintos
def good_group(x):
    return (len(x) >= 2) and (x["tiempo_años"].nunique() >= 2)

df_mlm = df_mlm.groupby("num.consec").filter(good_group)

# 3. Resetear índice para evitar huecos
df_mlm = df_mlm.reset_index(drop=True)

print("Observaciones:", len(df_mlm),
      "| Pacientes:", df_mlm["num.consec"].nunique())

# 4. Modelo lineal mixto (intercepto + pendiente aleatoria)
modelo = smf.mixedlm(
    formula="MDS_UPDRS_III ~ tiempo_años",
    data=df_mlm,
    groups=df_mlm["num.consec"],
    re_formula="~tiempo_años"        #  ← incluye random-slope
).fit(reml=False, method="lbfgs")     # LBFGS: más estable que default Newton

print(modelo.summary())

# 5. Extraer pendientes individuales (coef. de tiempo_años)
pend_slopes = {
    k: v["tiempo_años"] for k, v in modelo.random_effects.items()
}

df_slopes = (pd.Series(pend_slopes, name="pendiente_III")
               .reset_index()
               .rename(columns={"index": "num.consec"}))

display(df_slopes.head())


## Créditos

Desarrollado por:  
**Laboratorio Clínico de Enfermedades Neurodegenerativas (LCEN)**  
Instituto Nacional de Neurología y Neurocirugía “Manuel Velasco Suárez” (INNN)  
Ciudad de México, México  

Colaboradores:  
- Dr. Amin Cervantes-Arriaga  
- Dra. Mayela Rodríguez-Violante  
- Equipo ReMePARK – LCEN-INNN


## Licencia

Este trabajo se distribuye bajo la licencia [MIT](./LICENSE).  
Puedes usarlo, modificarlo y compartirlo citando la fuente original.
